# Still not working
To do
- Got to get requests for date and data

In [28]:
# Importing packages

import matplotlib.pyplot as plt
from datetime import datetime
import alpaca_trade_api as tradeapi
import numpy as np
import loader

#API keys in gitignored folder
from config import key_id, secret_key

In [29]:
#API keys, accounts, apis
api = tradeapi.REST(key_id,secret_key)
account = api.get_account()
api.list_positions()

[]

In [19]:
#custom backtester 
#Requires Orders to be in dict format


class positionHandler:
    def __init__(self,startingBalance=10000,liveTrading=False):
        self.cashBalance = startingBalance
        self.livePositions = {} # Dictionary of currently open positions
        self.openOrders = [] # List of open orders
        self.positionHistory = [] # List of items [Symbol, new position size]
        self.tradeHistory = [] # List of filled trades		
        self.liveTrading = liveTrading

        
    def placeOrder(self, symbol, quantity, side, orderType, time_in_force, limit_price=None, stop_price=None, client_order_id=None):

        if self.liveTrading:
            returned = api.submit_order(symbol, qty, side, orderType, time_in_force, limit_price, stop_price, client_order_id)
            self.tradeHistory.append(returned) # You'll probably want to make a simpler custom order dict format
        else:
            self.tradeHistory.append(order_dict) 
            #Change from dict to whatever format you use later

        if orderType == "market":
            try:
                if side == "buy":
                    fillPrice = data[symbol]["close"] # You'll need to make adjustments to the backtest fill price assumptions

                    self.livePositions[symbol][size] = self.livePositions[symbol][size] + quantity
                    self.cashBalance -= quantity * fillPrice
                elif side == "sell":
                    fillPrice = data[symbol]["close"]

                    self.livePositions[symbol][size] = self.livePositions[symbol][size] - quantity
                    self.cashBalance += quantity * fillPrice


                if self.livePositions[symbol][size] == 0:
                    del self.livePositions[symbol]

            except:
                self.livePositions[symbol] = {}

                if side == "buy":
                    self.livePositions[symbol][size] = quantity
                elif side == "sell":
                    self.livePositions[symbol][size] = -quantity

            self.positionHistory.append([symbol,self.livePositions[symbol]])
        else:
            self.openOrders.append(order_dict) # You'll probably want to make a simpler custom order dict format

    def processOpenOrders(self):

        for order in self.openOrders:
            symbol = order["symbol"]

            if self.liveTrading:
                returned = api.get_order(order["order_id"])

                # Process the live order status into your storage format as necessary

            else:
                # Historical data input has to be adjusted for your own data pipeline setup
                timeStepMin = data[symbol]["low"] # Reads the minimum trade price since last data point
                timeStepMax = data[symbol]["high"] # Reads the maximum trade price since last data point

                if order["orderType"] == "limit":
                    try:
                        if order["side"] == "buy" and order["limit"] > timeStepMin:
                            # You'll need to make adjustments to the backtest fill price assumptions
                            fillPrice = data[symbol]["close"] 

                            self.livePositions[symbol][size] = self.livePositions[symbol][size] + quantity
                            self.cashBalance -= quantity * fillPrice
                            self.positionHistory.append([symbol,self.livePositions[symbol]])
                        elif order["side"] == "sell" and order["limit"] < timeStepMax:
                            fillPrice = data[symbol]["close"]

                            self.livePositions[symbol][size] = self.livePositions[symbol][size] - quantity
                            self.cashBalance += quantity * fillPrice
                            self.positionHistory.append([symbol,self.livePositions[symbol]])

                    except:
                        self.livePositions[symbol] = {}

                        if order["side"] == "buy" and order["limit"] > timeStepMin:
                            fillPrice = data[symbol]["close"]

                            self.livePositions[symbol][size] = quantity
                            self.cashBalance -= quantity * fillPrice
                            self.positionHistory.append([symbol,self.livePositions[symbol]])
                        elif order["side"] == "sell" and order["limit"] < timeStepMax:
                            fillPrice = data[symbol]["close"]

                            self.livePositions[symbol][size] = -quantity
                            self.cashBalance += quantity * fillPrice
                            self.positionHistory.append([symbol,self.livePositions[symbol]])

                #elif 
                # Add any other processing for other required order types

                
    def returnOpenPosition(self,symbol):
        try:
            return self.livePositions[symbol]
        except:
            return 0

In [22]:
# Use position handler 
backtester = positionHandler(startingBalance=10000, liveTrading=False)

backtestSymbolList = ["SPY", "AAPL", "BBRY"]
positionSizing = 0.33

cashBalanceList = []

#Load Data
data=api.get_barset('AAPL', 'minute', limit=500).df
dateList = api.get_barset('AAPL', 'minute', limit=500).df


In [24]:
# It can be helpful to have a list of the dates associated with their
# respective bars

timeSteps =len(dateList)
barIterator = 0


In [25]:
while barIterator < timeSteps:
    data = data
    for symbol in backtestSymbolList:
        # Historical data input has to be adjusted for your own data pipeline
        # Simple moving average cross strategy
        price = data[symbol]["close"]
        SMA20 = data[symbol]["SMA20"]
        SMA50 = data[symbol]["SMA50"]

        if SMA20 > SMA50:
            openPosition = backtester.returnOpenPosition(symbol)

            if openPosition == 0:
                cashBalance = backtester.cashBalance

                # Calculates required position size
                targetPositionSize = int(cashBalance / (price / positionSizing))

                backtester.placeOrder(
                    symbol,
                    targetPositionSize,
                    "buy",
                    "market",
                    "gtc")  # Market order to open position

        else:
            openPosition = backtester.returnOpenPosition(symbol)

            # Market order to fully close position
            backtester.placeOrder(
                symbol, openPosition, "sell", "market", "gtc")
    cashBalance.append(backtester.cashBalance)

tradeHistory = backtester.tradeHistory
positionHistory = backtester.positionHistory
finalBalance = backtester.cashBalance


# Defines the plot for each trading symbol
f, ax = plt.subplots()
f.suptitle(symbol)

AttributeError: module 'loader' has no attribute 'get_data'

In [ ]:
timeList = []
for date in dateList:
    timeList.append(datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ'))
timeList = np.array(timeList)
# Plots market data and indicators
ax.plot(timeList, cashBalanceList, label=symbol, color="black")

# Add functions to analyse performance over time and calculate metrics